https://pypi.org/project/textstat/

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import metrics
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_auc_score ,roc_curve,auc
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from collections import Counter
import numpy as np
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix,precision_recall_fscore_support,accuracy_score
from sklearn import tree
import xgboost as xgb
from sklearn import preprocessing
from tqdm import tqdm_notebook as tqdm


import pickle
import csv
import pandas as pd

In [2]:
import os
import stanza # Stanford's stanza package
# stanza.download('en') # run this once
import pandas as pd
import numpy as np
import textstat


import subprocess
import shlex
from subprocess import Popen, PIPE

In [10]:
data = pd.read_csv("../data/LCA_CleanData_5.csv")

In [22]:
# data = data.iloc[1750:1752,:]
# data

In [12]:
processor_dict = {
    'tokenize': 'gsd', 
    'pos': 'bnc', 
    'lemma': 'default'
}

nlp = stanza.Pipeline('en', processors=processor_dict)

2021-03-12 22:52:27 WARNING: Can not find tokenize: gsd from official model list. Ignoring it.
2021-03-12 22:52:27 WARNING: Can not find pos: bnc from official model list. Ignoring it.
2021-03-12 22:52:27 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | ewt       |
| pos       | ewt       |
| lemma     | ewt       |
| depparse  | ewt       |
| sentiment | sstplus   |
| ner       | ontonotes |

/home/mostofa/anaconda3/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /opt/conda/conda-bld/pytorch_1607369981906/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
2021-03-12 22:52:27 INFO: Use device: cpu
2021-03-12 22:52:27 INFO: Loading: tokenize
2021-03-12 22:52:27 INFO: Load

In [13]:
result = []


for index, row in tqdm(data.iterrows(), total=data.shape[0]):
#     if index < 1553: continue
    
    text = row["text_modified"]
    text_list = text.split("<start>")
    text_list = text_list[:-1] if len(text_list) > 1 else text_list
    single_result = list(row)
    
    input_file = '../data/lemmatize_pos_sentences.txt'
    os.getcwd()
    
    new_without_start = "".join(text_list)
    try:
        doc = nlp(new_without_start)
    except:
        print('problem!', new_without_start)
        continue
    out = open(input_file, 'w')
    for sentence in doc.sentences:
        s = ''
        for word in sentence.words:
            s+='{}_{}'.format(word.lemma, word.xpos) + ' ' # needs to be xpos so it uses Penn Treebank
        out.write(s.strip()+'\n')
    out.close()
    output_file = '../data/lc_out.csv'
    command = 'cd ../lca;python lc.py {} > {}'.format(input_file, output_file)
    os.system(command)
    os.getcwd()
    lc = pd.read_csv(output_file)
    single_result.extend(list(lc.iloc[0]))
    
#     single_result = single_result + lexical_result
    single_word_file = [0]*10
    missing = []
    run_error = []
    for line in text_list:
        line  =line.strip()
#         print(line)
#         input_file = '../data/lemmatize_pos_sentences.tagged'
        doc = nlp(line)
#         print(len(doc.sentences))
        out = open(input_file, 'w')
        for sentence in doc.sentences:
            s = ''
            l = 0
            for word in sentence.words:
                s+='{} {}'.format(word.lemma, word.xpos) + ' ' # needs to be xpos so it uses Penn Treebank
                l+=1
            out.write('{} {}\n'.format(l, s.strip()))
        out.close()

        cmd = 'cd ../d-level-analyzer/COLLINS-PARSER;'
        cmd += 'gunzip -c models/model2/events.gz | code/parser ../{} models/model2/grammar 10000 1 1 1 1 > ../../data/parsed.m2;'.format(input_file)
        #print(cmd)
        cmd += 'cd ..;'
        cmd += 'python d-level.py ../data/parsed.m2 > ../data/dlevel.dla;'
        proc = subprocess.Popen(cmd, stdout=PIPE, stderr=PIPE, shell=True)
    #     print(proc.stdout.read(), proc.stderr.read())

        os.system(cmd)
        
        try: 
            d_level = pd.read_csv('../data/dlevel.dla')
        except: 
            run_error.append(line)
            continue
#         print(d_level.iloc[0])

        for i, item in enumerate(d_level.iloc[0][1:]):
            if i == item == 0:
                missing.append(line)
            single_word_file[i] += item

        
    missing = "<start> ".join(missing)
    run_error = "<start> ".join(run_error)
    single_result = single_result + [d_level.iloc[0][0]] + single_word_file + [missing] + [run_error]
    result.append(single_result)
    
#     columns = list(data.columns) + list(lc.columns.str.replace(' ','')) + list(d_level.columns.str.replace(' ',''))
    columns = list(data.columns) + list(lc.columns.str.replace(' ','')) + list(d_level.columns.str.replace(' ','')) + ["Missing"] + ['Run_error']
#     print(len(columns))
#     print(columns)
#     print(len(result))
    
    new_df = pd.DataFrame(result, columns=columns).reset_index()
    new_df.to_csv('../data/Features.csv', index=False)  
    

<ipython-input-13-0d1a81641d49>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for index, row in tqdm(data.iterrows(), total=data.shape[0]):


?
Imagine you are 14 years old but only as tall as a pencil. You don’t have any arms. When you turn 60, your first arm grows. Another arm grows, and then another. One day you realize you have nine arms. You are now 150 years old and as tall as a three-story building.
What if flowers grew from your head? What if your ribs were made of wood? And let’s say you were green with sharp spines. Does that seem strange to you? That’s what it would be like to be a saguaro.
Would you care to dance?
Can you say saguaro? Imagine that the gu is a w. Try saying it like this: suh-WAR-oh. Saguaro. suh-WAR-oh.
A saguaro (Carnegiea gigantea) is a giant green cactus with a round trunk and curved arms. People often think that saguaros look like humans. Many saguaros look as though they are waving, and some appear to be dancing. But despite how they look, they all stand silently through the day and night.
The saguaro cactus is the silent keeper of desert secrets. From above, this green giant witnesses many l

The saguaro even has a park named after it! Saguaro National Park near Tucson, Arizona, is the home of many beautiful saguaros. Visitors from all over the world come to see these mysterious giants. They take photographs of themselves with the saguaros. Some tourists like to hike on park trails and enjoy the beauty of the saguaro in their Sonoran desert home.
Today, there are laws to protect saguaros. In certain places you cannot dig them up without a permit. If you do, you can be fined. When new roads are built, saguaros are moved to keep them safe.
A photographer captures a saguaro at sunset.
These are just some of the secrets of the mighty saguaro. Other stories may never be known. But one day you might visit the Sonoran Desert. You might walk out into the desert’s silence and sit next to a saguaro. You might carefully touch its tough skin and stare up at its huge green arms. You might picture the animals that have eaten its fruit.
You might even scratch your head and try to guess it